## Part III (Static Replication)

## Data Handling

In [34]:
import datetime as dt
import pandas as pd
from pprint import pprint
import numpy as np
import pandas
from scipy.interpolate import interp1d
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad
import matplotlib.pylab as plt

In [35]:
# Get SPX data

spx_df = pd.read_csv('SPX_options.csv')
spx_df.head(3)

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E


In [36]:
# To SPX, add col: Mid Price

spx_df['mid_price'] = (spx_df['best_bid'] + spx_df['best_offer']) / 2
spx_df.head(3)

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid_price
0,20201201,20201218,C,100000,3547.6,3570.5,E,3559.05
1,20201201,20201218,C,200000,3447.6,3470.5,E,3459.05
2,20201201,20201218,C,300000,3347.7,3370.6,E,3359.15


In [37]:
# To SPX, add col: K

spx_df["K"] = spx_df["strike_price"]/1000
spx_df.head(3)

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid_price,K
0,20201201,20201218,C,100000,3547.6,3570.5,E,3559.05,100.0
1,20201201,20201218,C,200000,3447.6,3470.5,E,3459.05,200.0
2,20201201,20201218,C,300000,3347.7,3370.6,E,3359.15,300.0


In [38]:
# To SPX, add T (duration in years)

# Convert 'exdate' and 'date' columns to datetime
spx_df['exdate'] = pd.to_datetime(spx_df['exdate'], format='%Y%m%d')
spx_df['date'] = pd.to_datetime(spx_df['date'], format='%Y%m%d')

# Calculate the duration in years for each row
spx_df['T'] = (spx_df['exdate'] - spx_df['date']).dt.days / 365

spx_df.head(3) 

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid_price,K,T
0,2020-12-01,2020-12-18,C,100000,3547.6,3570.5,E,3559.05,100.0,0.046575
1,2020-12-01,2020-12-18,C,200000,3447.6,3470.5,E,3459.05,200.0,0.046575
2,2020-12-01,2020-12-18,C,300000,3347.7,3370.6,E,3359.15,300.0,0.046575


In [39]:
rates_df = pd.read_csv('zero_rates_20201201.csv')
rates_df.head(3)

,date,days,rate
0,20201201,7,0.102280
1,20201201,13,0.114128
2,20201201,49,0.216480


In [40]:
# SABR

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma


## Set Parameters

In [41]:
# Get Duration in Years: From 1-Dec-2020 To 15-Jan-2021

today = dt.date(2020, 12, 1)
expiry = dt.date(2021, 1, 15)
T = (expiry - today).days / 365
T

0.1232876712328767

In [42]:
# For SPX, Get nearest K to Spot of 3662.45

S = 3662.45
closest_strike_index = spx_df['K'].sub(S).abs().idxmin()
K = spx_df.iloc[closest_strike_index]['K']

K

3660.0

In [43]:
# Check SPX mid_price where K = 243 & T = T

spx_df[(spx_df["K"] == 3660) & (spx_df["T"] == T)]

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid_price,K,T
1109,2020-12-01,2021-01-15,C,3660000,94.5,95.4,E,94.95,3660.0,0.123288
1478,2020-12-01,2021-01-15,P,3660000,94.8,95.5,E,95.15,3660.0,0.123288


> From the output above, SPX has both Call & Put options matching K = 3660 & T = 0.123288 years

> From the output above, SPY has both Call & Put options matching K = 243 & T = 0.123288 years

In [44]:
price_SPX_call = 94.95
price_SPX_put = 95.15

In [63]:
# Get interpolated r for given duration T

days = rates_df['days'].values
rates = rates_df['rate'].values

# Create a linear interpolation function
interp_func = interp1d(days, rates, kind='linear')

# Interpolate the rate for 45 days
r = interp_func(T*365)/100

print(f"The interpolated 'r' is: {r}")

The interpolated 'r' is: 0.0020510755555555554


In [46]:
print(f"S = {S}, K = {K}, r = {r}, Call price = {price_SPX_call}, Put price = {price_SPX_put}, T = {T}.")

S = 3662.45, K = 3660.0, r = 0.0020510755555555554, Call price = 94.95, Put price = 95.15, T = 0.1232876712328767.


## Value via Black-Scholes Model

> Method 1: Use ATM Implied Volatility --> Value function

In [47]:
# Black-Scholes Call

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

# Black-Scholes Put

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*(T)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*(T))*norm.cdf(-d2) - S*norm.cdf(-d1)

In [48]:
# Define Implied Call Volatility from B-S Model

def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                            BlackScholesCall(S, K, r, x, T),
                            1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

# Define Implied Put Volatility from B-S Model

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [49]:

impliedVol_call = impliedCallVolatility(S=S, K=K, r=r, price=price_SPX_call, T=T)
impliedVol_put = impliedPutVolatility(S,K,r,price_SPX_put,T)

print(f"The Black-Scholes Implied Volatility for the Call is: {impliedVol_call}")
print(f"The Black-Scholes Implied Volatility for the Put is: {impliedVol_put}")

The Black-Scholes Implied Volatility for the Call is: 0.18188348370668755
The Black-Scholes Implied Volatility for the Put is: 0.18886028486825987


In [50]:
def value_bsm(S,r,T,sigma):
    integrand = lambda x:(((S**(1/3))*np.exp((1/3)*(r - (sigma**2)/2)*T + (1/3)*sigma*np.sqrt(T)*x) + 1.5*(np.log(S) + (r - (sigma**2)/2)*T + sigma*np.sqrt(T)*x) + 10) * np.exp(-(x**2)/2))
    value = np.exp(-r*T)/np.sqrt(2*np.pi) * quad(integrand, -(np.inf), np.inf)[0]
    return value

value_bs_1_call = value_bsm(S,r,T,impliedVol_call)
value_bs_1_put = value_bsm(S,r,T,impliedVol_put)

print(f"The Call Derivative value via Black-Scholes, using ATM Implied Volatility, is: {value_bs_1_call}")
print(f"The Put Derivative value via Black-Scholes, using ATM Implied Volatility, is: {value_bs_1_put}")

The Call Derivative value via Black-Scholes, using ATM Implied Volatility, is: 37.70523431801233
The Put Derivative value via Black-Scholes, using ATM Implied Volatility, is: 37.704449380920934


> Method 2: Use B-S Call & Put Prices -->  Get Model-Free Integrated Variance --> Value function

In [51]:
# Black Scholes Model Integrated Variance Function
def callintegrand_BSM(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price

def putintegrand_BSM(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

In [52]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand_BSM(x, S, r, T, impliedVol_put), 0.0, F)
I_call = quad(lambda x: callintegrand_BSM(x, S, r, T, impliedVol_call), F, 5000)
E_var_BSM = 2*np.exp(r*T)*(I_put[0] + I_call[0])
sigma_BSM = np.sqrt(E_var_BSM/T)

print(f"The Black-Scholes Integrated Variance is: {E_var_BSM}")
print(f"The Black-Scholes Model-Free Volatility is: {sigma_BSM}")

The Black-Scholes Integrated Variance is: 0.004242141853643472
The Black-Scholes Model-Free Volatility is: 0.18549523962624231


In [53]:
value_bs_2_SPX = value_bsm(S,r,T,sigma_BSM)

print(f"The Derivative value via Black-Scholes, using Model-Free Volatility, is: {value_bs_2_SPX}")

The Derivative value via Black-Scholes, using Model-Free Volatility, is: 37.704831656940435


## Pricing via Bachelier Model

> Method 1: Use Implied Volatility --> Value Function

In [54]:
# Bachelier Call

def BachelierCall(S, K, r, sigma, T):
    result = (S - K)/ (sigma *np.sqrt(T))
    return np.exp(-r*(T)) * ((S-K) * norm.cdf(result) + sigma * np.sqrt(T) * norm.pdf(result))

# Bachelier Put

def BachelierPut(S, K, r, sigma,T):
    result = (K - S)/ (sigma *np.sqrt(T))
    return np.exp(-r*(T)) * ((K - S) * norm.cdf(result) + sigma * np.sqrt(T) * norm.pdf(result))

In [55]:
# Execute S = 3662.45, K = 3660, T = 45/365 years, r = 0.0020510756

impliedVol_call = impliedCallVolatility(S=S, K=K, r=r, price=price_SPX_call, T=T)
impliedVol_put = impliedPutVolatility(S=S, K=K, r=r, price=price_SPX_put, T=T)

def value_b(S,r,T,sigma):
    integrand = lambda x:((S + sigma*np.sqrt(T)*x)**(1/3) + 1.5*np.log(S + sigma*np.sqrt(T)*x) + 10) * np.exp(-(x**2)/2)
    value = np.exp(-r*T)/np.sqrt(2*np.pi) * quad(integrand, -(np.inf), np.inf)[0]
    return value

value_b_1_call = value_b(S,r,T,impliedVol_call)
value_b_1_put = value_b(S,r,T,impliedVol_put)


print(f"The Call Derivative value via Bachelier, using ATM Implied Volatility, is: {value_b_1_call}")
print(f"The Put Derivative value via Bachelier, using ATM Implied Volatility, is: {value_b_1_put}")

The Call Derivative value via Bachelier, using ATM Implied Volatility, is: 37.71359681718632
The Put Derivative value via Bachelier, using ATM Implied Volatility, is: 37.71359681712779


> Method 2: Use Bachelier Call & Put Prices -->  Get Model-Free Integrated Variance --> Value function

In [56]:
# Black Scholes Model Integrated Variance Function
def callintegrand_B(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def putintegrand_B(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price

In [57]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand_B(x, S, r, T, impliedVol_put), 0.0, F)
I_call = quad(lambda x: callintegrand_B(x, S, r, T, impliedVol_call), F, 5000)
E_var_B = 2*np.exp(r*T)*(I_put[0] + I_call[0])
sigma_B = np.sqrt(E_var_B/T)


In [58]:
value_b_2 = value_b(S,r,T,sigma_B)

print(f"The Derivative value via Bachelier, using Integrated Variance, is: {value_b_2}")

The Derivative value via Bachelier, using Integrated Variance, is: 37.71359681793495


## Pricing via Static Replication

In [59]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [60]:
# Execute with SABR (45 days) values from Part II

alpha = 1.817
beta = 0.7
rho = -0.404
nu = 2.790

F = S * np.exp(r*T)
I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
E_var_SABR = 2*np.exp(r*T)*(I_put[0] + I_call[0])
sigma_SABR = np.sqrt(E_var_SABR/T)

print(f"The SABR Integrated Variance is: {E_var_SABR}")
print(f"The SABR Model-Free Volatility is: {sigma_SABR}")

The SABR Integrated Variance is: 0.006337324864089065
The SABR Model-Free Volatility is: 0.2267217372018694


In [61]:
# express h(F)
hF = F**(1/3) + 1.5*np.log(F) + 10

# express h''(F)
hF_dbl_prime = -(2/9)*F**(-5/3) - 1.5/(F**2)

# express put & call integrands
def put_integrand_CM(F,S,K,r,sigma,T): 
    integrand = hF_dbl_prime * BlackScholesPut(F,K,r,sigma,T)
    return integrand

def call_integrand_CM(F,S,K,r,sigma,T): 
    integrand = hF_dbl_prime * BlackScholesCall(F,K,r,sigma,T)
    return integrand


I_put = quad(lambda x:put_integrand_CM(F,S,x,r,sigma_SABR,T),0,F)
I_call = quad(lambda x:call_integrand_CM(F,S,x,r,sigma_SABR,T),F,5000)

value_SR_1 = np.exp(-r*T)*hF + I_put[0] + I_call[0]

print(f"The Derivative value via Static Replication, using Black-Scholes for the Call & Put prices is: {value_SR_1}")

The Derivative value via Static Replication, using Black-Scholes for the Call & Put prices is: 37.699613637477576


In [62]:
# express h(F)
hF = F**(1/3) + 1.5*np.log(F) + 10

# express h''(F)
hF_dbl_prime = -(2/9)*F**(-5/3) - 1.5/(F**2)

# express put & call integrands
def put_integrand_CM(F,S,K,r,sigma,T): 
    integrand = hF_dbl_prime * BachelierPut(F,K,r,sigma,T)
    return integrand

def call_integrand_CM(F,S,K,r,sigma,T): 
    integrand = hF_dbl_prime * BachelierCall(F,K,r,sigma,T)
    return integrand


I_put = quad(lambda x:put_integrand_CM(F,S,x,r,sigma_SABR,T),0,F)
I_call = quad(lambda x:call_integrand_CM(F,S,x,r,sigma_SABR,T),F,5000)

value_SR_2 = np.exp(-r*T)*hF + I_put[0] + I_call[0]

print(f"The Derivative value via Static Replication, using Bachelier for the Call & Put prices is: {value_SR_2}")

The Derivative value via Static Replication, using Bachelier for the Call & Put prices is: 37.71527504035332
